Text Processing: Clean and preprocess the extracted HTML data:

Remove HTML tags and scripts.
Normalize text (lowercasing, removing punctuation).
Tokenization and stop-word removal can be beneficial.


Feature Extraction: Identify and extract relevant features that could help classify a domain as an eCommerce store:

Keywords and phrases.
HTML structure elements (e.g., presence of <product>, <price>, <cart> tags).
Meta tags relevant to eCommerce.


LLM Utilization:

Use large language models (like GPT-3, BERT) to classify extracted text or assist in feature selection. Fine-tuning these models on your labeled data could yield excellent results.

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import requests
import warnings
from urllib.parse import urljoin
import re
from googletrans import Translator
import nltk
import time
import requests
from urllib.parse import urlencode
from deep_translator import GoogleTranslator
import pandas as pd
import unicodedata
import spacy
import numpy as np
import emoji



In [2]:
data_1=pd.read_csv('noshop.csv')

In [3]:
data_1.shape

(12045, 5)

In [5]:
# # Phrases to search for
# phrases_to_search = [
#     "Terms and conditions",
#     "Privacy policy",
#     "Return and exchange policy",
#     "Place a return or exchange request",
#     "Track your order",
#     "Add to cart",
#     "add to cart",
#     "select product",
#     "Online shop",
#     "Shipping & Payment",
#     "Exchanges & Refunds",
#     "How to Shop",
#     "Order Status",
#     "Shipping Method",
#     "Trackin order",
#     "User Guide ",
#     "online e-commerce",
#     "Quick buy",
#     "Easy Return Guidance",
#     "Shipping Details",
#     "Online shop",
#     "Delivery",
#     "RETURNS AND EXCHANGE",  
#     "Payment methods",
#     "Returns and refunds",
#     "Shipping Information & Costs",
#     "Shopping basket",
#     "Loyalte program",
#     "Gift Cards",
#     "Shipping and payment",
#     "Returns / Product exchange",
#     "Payment", 
#     "Shipping & Delivery",
#     "Exchanges & Returns",
#     "Shipping and Exchange Policy",
#     "Shopping basket",
#     "Refund policy",
#     "All about shopping",
#     "Basket",
#     "Shopping cart",
#     "Return procedure",
#     "Payment methods",
#     "Guarantees and returns",
#     "Return/Complaint Instructions",
#     "About Click & Collect", # "https://www.bike-online.jp/hpgen/HPB/entries/43.html"
#     "E-shop",
#     "order status "
#     ]

In [6]:
# # Suppress SSL warnings
# # warnings.filterwarnings("ignore", category=requests.packages.urllib3.exceptions.InsecureRequestWarning)
# warnings.simplefilter(action='ignore')
# # Function to fetch HTML content from the domain
# def fetch_html(domain):
#     headers = {
#         'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
#     }
#     try:
#         response = requests.get(f'https://{domain}', headers=headers, timeout=10, verify=False)
#         if response.status_code == 200:
#             return response.text
#         else:
#             return ""
#     except requests.RequestException:
#         return ""

# # Function to extract and clean text from the HTML
# def extract_text_from_domain(domain):
#     # Fetch HTML content using fetch_html
#     html = fetch_html(domain)
#     if not html:
#         return "Error fetching the webpage."

#     try:
#         soup = BeautifulSoup(html, "html.parser")

#         # Extract text from all elements and clean it
#         text = soup.get_text(separator=" ")
#         cleaned_text = re.sub(r"\s+", " ", text).strip()

#         return cleaned_text
#     except Exception as e:
#         return f"Error extracting text: {e}"

# # Function to search for specific phrases
# def search_phrases(html_content, phrases):
#     if not isinstance(html_content, str):
#         return ''
#     found_phrases = []
#     for phrase in phrases:
#         if phrase.lower() in html_content.lower():
#             found_phrases.append(phrase)
#     return ', '.join(found_phrases)

# # Function to translate text
# def translate_text(text, src_language):
#     translator = Translator()
#     translation = translator.translate(text, src=src_language, dest="en")

# # Function to detect the HTML language
# def detect_language(html_content):
#     if not html_content:
#         return None
    
#     # Use regex to find the lang attribute directly
#     lang_match = re.search(r'<html[^>]*\blang=["\']?([a-zA-Z-]+)', html_content, re.IGNORECASE)
#     if lang_match:
#         detected_lang = lang_match.group(1).split('-')[0]  # Extract primary language
#         return detected_lang

#     # Fallback to BeautifulSoup parsing
#     soup = BeautifulSoup(html_content, 'lxml')
#     html_tag = soup.html
#     if html_tag and html_tag.get('lang'):
#         detected_lang = html_tag.get('lang').split('-')[0]
#         return detected_lang
#     return ""

# # Apply functions to DataFrame
# data_1['HTML'] = data_1['domain'].apply(fetch_html)
# data_1['Text'] = data_1['domain'].apply(extract_text_from_domain)
# data_1['found_phrases'] = data_1['HTML'].apply(lambda html: search_phrases(html, phrases_to_search))
# data_1['language'] = data_1['HTML'].apply(detect_language)
# data_1.head(4)

Cleaning functions:

-  Cleaning:

In [4]:
data_1 = data_1[data_1['HTML'].apply(lambda x: pd.notna(x) and x not in [[], ""])]
data_1 = data_1[data_1["Text"].apply(lambda x: not isinstance(x, float))]


In [5]:
data_1.shape

(9744, 5)

In [6]:
dict_symbols_to_change = {
    "\u2714": ' ',  # ✔
    "\u2605": ' ',  # ★
    "\u00A9": ' ',  # ©
    "\u00AE": ' ',  # ®
    "\u2022": ' ',  # •
    "\u2798": ', ',  # ➨
    "\u27BD": ', ',  # ➽
    "\u2661": ', ',  # ♡
    "\u266A": ', ',  # ♪
    "\u2026": ', ',  # …
    "\u25CF": ', ',  # ●
    "\u2122": ' ',  # ™
    "\u2713": ' ',  # ✓
    "\u2600": ' ',  # ☀ (Stars and Sun)
    "\u2665": ' ',  # ♥ 
    "\U000025BA": ' ', # ►
}

In [7]:
def replace_symbols(text):
    """
    Replace some symbols in text as defined in lookup.dict_symbols_to_change.
    """
    for key, value in dict_symbols_to_change.items():
        text = text.replace(key, value)
    return text


In [8]:
def remove_emojis(text):
    return emoji.replace_emoji(text, replace='')  # Replace emojis with empty string


In [9]:
data_1["Text"]=data_1["Text"].apply(replace_symbols)
data_1["Text"]=data_1["Text"].apply(remove_emojis)

In [10]:
data_1["len"]=data_1["Text"].apply(len)
data_1.sort_values("len",ascending=False).head(5)

,domain,HTML,Text,found_phrases,language,len
695,arena.pl,"<!DOCTYPE html><html lang=""pl""><head><link rel...",Arena.pl Platforma zakupowa Bezpieczne zak...,"Delivery, Payment, Basket, E-shop",pl,1079239
3860,fomosa.co.za,"<!DOCTYPE html> <html lang=""en-US""> <head><scr...",FOMO | Best Deals on Activities & Experiences ...,"Terms and conditions, Privacy policy, Delivery...",en,293642
1402,blumenbilder.org,"<!DOCTYPE html>\n<!--[if IE 6]>\n<html id=""ie6...",Leben mit Blumen | Blumenbilder.org IM TREND: ...,Payment,de,256146
614,anzlitlovers.com,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T...",ANZ LitLovers LitBlog | For lovers of Australi...,"Delivery, Payment",en,241540
3854,fogyas.info,"\r\n<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4....",Fogyás A sikeres fogyás titka Érezd jól ma...,NaN,hu,218860


In [11]:
data_1 = data_1[(data_1["len"] >= 1000) & (data_1["len"] <= 100000)]

In [ ]:
# data_1=data_1.drop(columns=["HTML","len"])

In [12]:
import html
import re
import unicodedata

def normalize_text(text):
    if not isinstance(text, str):  # Ensure input is a string
        return text
    
    # Decode HTML entities
    text = html.unescape(text)
    
    # Normalize Unicode (preserve foreign letters, remove accents)
    text = ''.join(c for c in unicodedata.normalize('NFKC', text) if unicodedata.category(c) != 'Mn')

    # Remove control characters (except line breaks)
    text = re.sub(r'[\x00-\x08\x0B\x0C\x0E-\x1F\x7F]', '', text)  

    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    
    return text.strip()

# Apply normalization function
data_1['normalized_text'] = data_1['Text'].astype(str).apply(normalize_text)

# Display the result
data_1.head(3)


,domain,HTML,Text,found_phrases,language,len,normalized_text
0,0101shop.com,<!DOCTYPE html>\r\n<html>\r\n<head>\r\n <me...,å åç ´è§£æ¸¸æ-2020å åç ´è§£æ¸¸æä¸è½...,NaN,NaN,12913,å åç è§£æ æ-2020å åç è§£æ æä è1⁄21⁄...
6,1001cartes.com,"<!DOCTYPE HTML>\n<html lang=""fr"">\n\t<head>\n\...","Faire-part de naissance, mariage, baptême - vo...","Add to cart, add to cart, Shipping Method, Pay...",fr,7773,"Faire-part de naissance, mariage, baptême - vo..."
7,1001cuponesdedescuento.cl,"<!DOCTYPE html>\n<html lang=""es"">\n<head>\n<me...",Cupones de descuento en Chile hasta 80% OFF en...,"Delivery, E-shop",es,13628,Cupones de descuento en Chile hasta 80% OFF en...


In [13]:
data_1.shape

(8641, 7)

In [14]:
import re

def remove_consecutive_duplicates(text):
    return re.sub(r'\b(\w+)(\s+\1)+\b', r'\1', text)


In [15]:
data_1['normalized_text']=data_1['normalized_text'].apply(remove_consecutive_duplicates)
data_1

,domain,HTML,Text,found_phrases,language,len,normalized_text
0,0101shop.com,<!DOCTYPE html>\r\n<html>\r\n<head>\r\n <me...,å åç ´è§£æ¸¸æ-2020å åç ´è§£æ¸¸æä¸è½...,NaN,NaN,12913,å åç è§£æ-2020å åç è§£æä è1⁄21⁄2-01...
6,1001cartes.com,"<!DOCTYPE HTML>\n<html lang=""fr"">\n\t<head>\n\...","Faire-part de naissance, mariage, baptême - vo...","Add to cart, add to cart, Shipping Method, Pay...",fr,7773,"Faire-part de naissance, mariage, baptême - vo..."
7,1001cuponesdedescuento.cl,"<!DOCTYPE html>\n<html lang=""es"">\n<head>\n<me...",Cupones de descuento en Chile hasta 80% OFF en...,"Delivery, E-shop",es,13628,Cupones de descuento en Chile hasta 80% OFF en...
8,100comments.com,"<!doctype html>\n<html lang=""en-US"">\n<head>\n...","Product Reviews, Free Samples, New Products @ ...",Privacy policy,en,7670,"Product Reviews, Free Samples, New Products @ ..."
9,100sp.ru,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ...",100sp — интернет-магазин. Товары по выгодным ц...,Delivery,ru,28787,100sp — интернет-магазин. Товары по выгодным ц...
...,...,...,...,...,...,...,...
12034,zuzu.deals,"<!DOCTYPE html>\r\n<html dir=""rtl"" lang=""h...",ZUZU DEALS - הקופונים והמבצעים הטובים ברשת! | ...,Payment,he,11552,ZUZU DEALS - הקופונים והמבצעים הטובים ברשת! | ...
12036,zwift.com,"<!DOCTYPE html><html lang=""de"" data-locale=""de...",Zwift | Die Indoor-Radfahrapp für Smart Traine...,Payment,de,3880,Zwift | Die Indoor-Radfahrapp für Smart Traine...
12037,zwiftinsider.com,"<!doctype html >\r\n<html lang=""en-US"">\r\n<he...","Zwift Insider | News, tips, and reviews for Zw...",Privacy policy,en,6695,"Zwift Insider | News, tips, and reviews for Zw..."
12039,zwrotnikraka.pl,"<!DOCTYPE html>\n<!--[if IE 6]>\n<html id=""ie6...",Zwrotnikraka.pl - portal onkologiczny dla Pacj...,NaN,pl,13295,Zwrotnikraka.pl - portal onkologiczny dla Pacj...


In [20]:
# data_1["lang_len"]=data_1["language"].astype(str).apply(len)
# data_1 = data_1[data_1["lang_len"] == 2]


In [21]:
data_1.head(5)

,domain,Text,found_phrases,language,normalized_text
0,0101shop.com,å åç ´è§£æ¸¸æ-2020å åç ´è§£æ¸¸æä¸è½...,NaN,NaN,å åç è§£æ-2020å åç è§£æä è1⁄21⁄2-01...
6,1001cartes.com,"Faire-part de naissance, mariage, baptême - vo...","Add to cart, add to cart, Shipping Method, Pay...",fr,"Faire-part de naissance, mariage, baptême - vo..."
7,1001cuponesdedescuento.cl,Cupones de descuento en Chile hasta 80% OFF en...,"Delivery, E-shop",es,Cupones de descuento en Chile hasta 80% OFF en...
8,100comments.com,"Product Reviews, Free Samples, New Products @ ...",Privacy policy,en,"Product Reviews, Free Samples, New Products @ ..."
9,100sp.ru,100sp — интернет-магазин. Товары по выгодным ц...,Delivery,ru,100sp — интернет-магазин. Товары по выгодным ц...


In [22]:

def decode_last(text):
    encoded_text = text

    # Using UTF-8 encoding and ignoring errors
    decoded_text = encoded_text.encode('latin1', errors='ignore').decode('utf-8', errors='ignore')
    return decoded_text


In [23]:
data_1["Decode"]=data_1["normalized_text"].apply(decode_last)

data_1.head(4)

,domain,Text,found_phrases,language,normalized_text,Decode
0,0101shop.com,å åç ´è§£æ¸¸æ-2020å åç ´è§£æ¸¸æä¸è½...,NaN,NaN,å åç è§£æ-2020å åç è§£æä è1⁄21⁄2-01...,卓 解戏-2020 卓 解戏 1212-0101手 12 0101手 1214 新的手o ...
6,1001cartes.com,"Faire-part de naissance, mariage, baptême - vo...","Add to cart, add to cart, Shipping Method, Pay...",fr,"Faire-part de naissance, mariage, baptême - vo...","Faire-part de naissance, mariage, baptme - vos..."
7,1001cuponesdedescuento.cl,Cupones de descuento en Chile hasta 80% OFF en...,"Delivery, E-shop",es,Cupones de descuento en Chile hasta 80% OFF en...,Cupones de descuento en Chile hasta 80% OFF en...
8,100comments.com,"Product Reviews, Free Samples, New Products @ ...",Privacy policy,en,"Product Reviews, Free Samples, New Products @ ...","Product Reviews, Free Samples, New Products @ ..."


___________________


# Text Translation pipeline


In [24]:
# import spacy
# import pandas as pd

# # Load the model once globally
# nlp = spacy.load("xx_ent_wiki_sm")
# nlp.add_pipe('sentencizer')

# def Tokenizer(text):
#     """Tokenize a text into sentences using spaCy."""
#     if pd.isna(text) or not isinstance(text, str):  # Handle NaN or non-string values
#         return []
    
#     doc = nlp(text)
#     return [sent.text.strip() for sent in doc.sents]  # Return a list of sentences

# # Apply the function to the 'text' column
# data_1['tokenized_sentences'] = data_1['normalized_text'].apply(Tokenizer)

# data_1.head(4)



In [16]:
from langdetect import detect

def detect_language(text):
    return detect(text)


In [17]:
data_1["detected_language"]=data_1["normalized_text"].apply(detect_language)

In [18]:
data_1.to_csv("cleaned_nonshop.csv", index=False)

In [27]:
sum_mismatches = 0
for i in range(len(data_1)):  
    if data_1.iloc[i]["language"] != data_1.iloc[i]["detected_language"]:  
        sum_mismatches += 1 
print(sum_mismatches)


1464


______

Failed translation model:

In [28]:
lang_map = {
    "af": "afr_Latn",
    "ak": "aka_Latn",
    "am": "amh_Ethi",
    "ar": "arb_Arab",
    "as": "asm_Beng",
    "ay": "ayr_Latn",
    "az": "azj_Latn",
    "bm": "bam_Latn",
    "be": "bel_Cyrl",
    "bn": "ben_Beng",
    "bho": "bho_Deva",
    "bs": "bos_Latn",
    "bg": "bul_Cyrl",
    "ca": "cat_Latn",
    "ceb": "ceb_Latn",
    "cs": "ces_Latn",
    "ckb": "ckb_Arab",
    "tt": "crh_Latn",
    "cy": "cym_Latn",
    "da": "dan_Latn",
    "de": "deu_Latn",
    "el": "ell_Grek",
    "en": "eng_Latn",
    "eo": "epo_Latn",
    "et": "est_Latn",
    "eu": "eus_Latn",
    "ee": "ewe_Latn",
    "fa": "pes_Arab",
    "fi": "fin_Latn",
    "fr": "fra_Latn",
    "gd": "gla_Latn",
    "ga": "gle_Latn",
    "gl": "glg_Latn",
    "gn": "grn_Latn",
    "gu": "guj_Gujr",
    "ht": "hat_Latn",
    "ha": "hau_Latn",
    "he": "heb_Hebr",
    "hi": "hin_Deva",
    "hr": "hrv_Latn",
    "hu": "hun_Latn",
    "hy": "hye_Armn",
    "nl": "nld_Latn",
    "ig": "ibo_Latn",
    "ilo": "ilo_Latn",
    "id": "ind_Latn",
    "is": "isl_Latn",
    "it": "ita_Latn",
    "jv": "jav_Latn",
    "ja": "jpn_Jpan",
    "kn": "kan_Knda",
    "ka": "kat_Geor",
    "kk": "kaz_Cyrl",
    "km": "khm_Khmr",
    "rw": "kin_Latn",
    "ko": "kor_Hang",
    "ku": "kmr_Latn",
    "lo": "lao_Laoo",
    "lv": "lvs_Latn",
    "ln": "lin_Latn",
    "lt": "lit_Latn",
    "lb": "ltz_Latn",
    "lg": "lug_Latn",
    "lus": "lus_Latn",
    "mai": "mai_Deva",
    "ml": "mal_Mlym",
    "mr": "mar_Deva",
    "mk": "mkd_Cyrl",
    "mg": "plt_Latn",
    "mt": "mlt_Latn",
    "mni-Mtei": "mni_Beng",
    "mni": "mni_Beng",
    "mn": "khk_Cyrl",
    "mi": "mri_Latn",
    "ms": "zsm_Latn",
    "my": "mya_Mymr",
    "no": "nno_Latn",
    "ne": "npi_Deva",
    "ny": "nya_Latn",
    "om": "gaz_Latn",
    "or": "ory_Orya",
    "pl": "pol_Latn",
    "pt": "por_Latn",
    "ps": "pbt_Arab",
    "qu": "quy_Latn",
    "ro": "ron_Latn",
    "ru": "rus_Cyrl",
    "sa": "san_Deva",
    "si": "sin_Sinh",
    "sk": "slk_Latn",
    "sl": "slv_Latn",
    "sm": "smo_Latn",
    "sn": "sna_Latn",
    "sd": "snd_Arab",
    "so": "som_Latn",
    "es": "spa_Latn",
    "sq": "als_Latn",
    "sr": "srp_Cyrl",
    "su": "sun_Latn",
    "sv": "swe_Latn",
    "sw": "swh_Latn",
    "ta": "tam_Taml",
    "te": "tel_Telu",
    "tg": "tgk_Cyrl",
    "tl": "tgl_Latn",
    "th": "tha_Thai",
    "ti": "tir_Ethi",
    "ts": "tso_Latn",
    "tk": "tuk_Latn",
    "tr": "tur_Latn",
    "ug": "uig_Arab",
    "uk": "ukr_Cyrl",
    "ur": "urd_Arab",
    "uz": "uzn_Latn",
    "vi": "vie_Latn",
    "xh": "xho_Latn",
    "yi": "ydd_Hebr",
    "yo": "yor_Latn",
    "zh-CN": "zho_Hans",
    "zh": "zho_Hans",
    "zh-TW": "zho_Hant",
    "zu": "zul_Latn",
    "pa": "pan_Guru"
}


In [29]:
# import torch
# from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
# import re

# # Load NLLB model and tokenizer
# model_name = "facebook/nllb-200-distilled-600M"
# device = "cuda" if torch.cuda.is_available() else "cpu"

# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

# def chunk_text(text, chunk_size=1000):
#     """Splits text into chunks of specified size, preserving sentences where possible."""
#     sentences = re.split(r'(?<=[.!?]) +', text)  # Split while keeping punctuation-based structure
#     chunks = []
#     current_chunk = []
#     current_length = 0
    
#     for sentence in sentences:
#         if current_length + len(sentence) + 1 > chunk_size:
#             chunks.append(" ".join(current_chunk))
#             current_chunk = []
#             current_length = 0
#         current_chunk.append(sentence)
#         current_length += len(sentence) + 1
    
#     if current_chunk:
#         chunks.append(" ".join(current_chunk))
    
#     return chunks

# def translate_chunks(text, src_lang, batch_size=4):
#     """Translates text efficiently by processing in batches, preserving non-text elements."""
#     if not text.strip():
#         return "[Empty content]"
    
#     if src_lang == "en":
#         return text  # Skip translation if language is English
    
#     src_lang_nllb = lang_map.get(src_lang, None)
#     if not src_lang_nllb:
#         return "[Unsupported language]"
    
#     chunks = chunk_text(text)
#     translated_chunks = []
    
#     for i in range(0, len(chunks), batch_size):
#         batch = chunks[i:i+batch_size]
#         inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True).to(device)
#         translated_tokens = model.generate(
#             **inputs,
#             max_length=500,  # Increased limit for longer outputs
#             num_beams=2,  # More beams for better quality
#             forced_bos_token_id=tokenizer.convert_tokens_to_ids(["eng_Latn"])[0]
#         )
#         translations = [tokenizer.decode(t, skip_special_tokens=True) for t in translated_tokens]
#         translated_chunks.extend(translations)
    
#     return " ".join(translated_chunks)

# # Apply translation with batch processing, skipping English rows
# data_1["translated_text"] = data_1.apply(
#     lambda row: row["normalized_text"] if row["language"] == "en" else translate_chunks(row["normalized_text"], row["language"]), axis=1
# )

# # Display final DataFrame
# data_1.head(4)


Successful transaltion model:

In [30]:
## Version I

# from deep_translator import GoogleTranslator
# import pandas as pd
# import ast  # To safely convert string representation of lists

# # Function to split long texts into chunks
# def chunk_text(text, chunk_size=2000):
#     return [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

# # Function to translate tokenized sentences
# def translate_sentences(tokenized_list, src_lang="auto", target_lang="en"):
#     if isinstance(tokenized_list, list):
#         text = " ".join(tokenized_list)  # Convert list to string
#     else:
#         text = str(tokenized_list)  # Ensure it's a string
    
#     # Split text into smaller chunks
#     chunks = chunk_text(text)
    
#     # Translate each chunk separately
#     translated_chunks = [
#         GoogleTranslator(source=src_lang, target=target_lang).translate(chunk) for chunk in chunks
#     ]
    
#     return " ".join(translated_chunks)

# # Apply translation conditionally
# def apply_translation(row):
#     if row["language"] == "en":
#         return row["tokenized_sentences"]
#     else:
#         return translate_sentences(row["tokenized_sentences"])

# # Assuming data_1 is a Pandas DataFrame
# data_1["translated_text_2"] = data_1.apply(apply_translation, axis=1)


In [31]:
# from deep_translator import GoogleTranslator
# import pandas as pd
# import ast  # To safely convert string representation of lists


# # Function to split long texts into chunks
# def chunk_text(text, chunk_size=4000):
#     # Split the text into smaller chunks
#     return [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

# # Function to translate tokenized sentences
# def translate_sentences(tokenized_list, src_lang="auto", target_lang="en"):
   
        
#         # Split text into smaller chunks
#         chunks = chunk_text(tokenized_list)
        
#         # Translate each chunk separately
#         translated_chunks = [
#             GoogleTranslator(source=src_lang, target=target_lang).translate(chunk) for chunk in chunks
#         ]
        
#         # Recombine the translated chunks into one string
#         return " ".join(translated_chunks)
    


# # Apply translation to tokenized sentences
# snippet["translated_text"] = snippet["normalized_text"].apply(translate_sentences)

# snippet.head(4)


In [32]:
# from deep_translator import GoogleTranslator
# import pandas as pd
# import ast  # To safely convert string representation of lists


# # Function to split long texts into chunks
# def chunk_text(text, chunk_size=2000):
#     # Split the text into smaller chunks
#     return [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

# # Function to translate tokenized sentences
# def translate_sentences(tokenized_list, src_lang="auto", target_lang="en"):
#     # try:
#     #     # Convert string to list if needed
#     #     if isinstance(tokenized_list, str):
#     #         tokenized_list = ast.literal_eval(tokenized_list)
            
#     #     # Join tokenized sentences into a single string
#     #     text_to_translate = " ".join(tokenized_list)
        
#         # Split text into smaller chunks
#         chunks = chunk_text(tokenized_list)
        
#         # Translate each chunk separately
#         translated_chunks = [
#             GoogleTranslator(source=src_lang, target=target_lang).translate(chunk) for chunk in chunks
#         ]
        
#         # Recombine the translated chunks into one string
#         try:
#             # Your translation logic
#             return " ".join(translated_chunks)
#         except Exception as e:
#             print(f"Error translating: {e}")  # Debugging
#             return "Translation Failed"

    
#     # except Exception as e:
#     #     return f"Error: {e}"

# # Apply translation to tokenized sentences
# data_1["translated_text"] = data_1["normalized_text"].apply(translate_sentences)

# data_1.head(4)


Translation attemt vol 2

In [33]:
import warnings
warnings.simplefilter(action="ignore")
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

article_fr = """Купить Шубуиз Натурального Меха: Лучшие Цены в Киеве и Украине | Chicly-Furs Рус Укр МАГАЗИН В КИЕВЕ Контакты О нас Доставка и оплата Обмен и возврат Советы Отзывы Вход +38 (067) 440 47 00 Перезвонить вам? Перезвонить вам? Укажите ваш номер телефона и имя. Мы свяжемся с вами в ближайшее время. Имя Телефон Отправить 0 Добавьте товары в желания 0 Добавьте товары для сравнения График работы: 12:00 - 19:00 0 Мой заказ ПАЛЬТО Все пальто Пальто демисезонные Пальто зимние женские Все зимние пальто Зимние пальто с мехом Кашемировые пальто Пальто с мехом Плащи Пальто с капюшоном НАТУРАЛЬНЫЕ ШУБЫ Все шубы Шубы норковые Все норковые шубы Полушубки норковые Норковые шубы с капюшоном Трансформеры из норки Элитная норка Норковые шубы больших размеров Шубы из голубой норки Шубы-автоледи из норки Шубы из мутона Все шубы из мутона Полушубки из мутона Полушубки Шубы из нутрии Все шубы из нутрии Трансформеры из нутрии Полушубки из нутрии Шубы из песца Все шубы из песца Трансформеры из песца Полушубок из песца Меховые кардиганы Шубы из бобрика (кролика рэкс) Все шубы из кролика рэкса Трансформеры из кролика Полушубки из бобрика (кролика рэкс) Трансформеры Шубы из кролика Все шубы из кролика Полушубки из кролика Шубы из куницы Все шубы из куницы Полушубки из куницы Шубы из каракуля Шубы из чернобурки Все шубы из чернобурки Полушубки из чернобурки Трансформеры из чернобурки Шубы из бобра Шубы из редких мехов Все шубы с эксклюзивного меха Трансформеры из редких мехов Полушубки из эксклюзивных мехов Шубы из лисы Все шубы из лисы Трансформеры из лисы Полушубки из лисы КУРТКИ Все куртки Дубленки натуральные Эко дубленки Зимние куртки Меховые куртки Куртки-дутики Куртки из экокожи ШУБЫ ИЗ ЭКО-МЕХА Шубы из эко-меха зимние Демисезонные искусственные шубы Куртки из эко-меха ЖИЛЕТКИ Все жилетки Жилетки из песца Жилетки из чернобурки Жилетки из норки Жилетки из кролика Жилетки из лисы Кашемировые жилеты с мехом Жилетки из кашемира и дутики Жилетки из редких мехов Жилетки из ламы Жилетки из шиншиллового кролика ПУХОВИКИ Пуховики зимние Пуховики осень-весна ПАРКА Парки женские Все женские парки Парки зимние АКСЕССУАРЫ Перчатки женские Головные уборы Шапки меховые женские Платки меховые Сумки Меховые накидки Шарфы меховые женские Пояса Чехлы для верхней одежды ОДЕЖДА Вся одежда Кардиганы и кофты Платья Спортивные костюмы Костюмы Юбки Гольфы и свитера Джинсы Пиджаки и рубашки Брюки и леггинсы Сарафаны и топы   2012—2025 "ШИКАРНЫЕ МЕХА" Все тексты и графические материалы на сайте являются собственностью компании, копирование без уведомления и любое их использование преследуется по Закону Украины. Мобильная версия Каталог ПАЛЬТО НАТУРАЛЬНЫЕ ШУБЫ КУРТКИ ШУБЫ ИЗ ЭКО-МЕХА ЖИЛЕТКИ ПУХОВИКИ ПАРКА АКСЕССУАРЫ ОДЕЖДА """
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

# translate French to English
tokenizer.src_lang = "ru_XX"
encoded_fr = tokenizer(article_fr, return_tensors="pt")
generated_tokens = model.generate(**encoded_fr, forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"])
translation =tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
print(translation)
# => "Hello, how are you?"


["The name Phone Send 0 Add Goods to Wishes 0 Add Goods to Compare Working Time: 12:00 - 19:00 0 My Order PALTY All Coats De-season Coat Winter Women's Coat Winter Coat Coat Casual Coat Casual Coat Plate Pallet with Capuchon PAllet with Capuchon NATURAL FISHERIES All Shoes All Shoes All Shoes of the Horses All Shoes of the Horses All Shoes of the Northern Territory All Shoes of the Northern Territory The Shoes of the Northern Territory The Shoes of the Northern Territory The Shoes of the Capuchon PAllet with Capuchon Transformers of the Northern Territory The Highest Prices in Kyiv and Ukraine The Shoes of the Blue-nose Shoe Shoe-drivers of the Northern Territory The Shoes of the Muton The Shoes of the"]


In [34]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast


article_ar = """Купить Шубуиз Натурального Меха: Лучшие Цены в Киеве и Украине | Chicly-Furs Рус Укр МАГАЗИН В КИЕВЕ Контакты О нас Доставка и оплата Обмен и возврат Советы Отзывы Вход +38 (067) 440 47 00 Перезвонить вам? Перезвонить вам? Укажите ваш номер телефона и имя. Мы свяжемся с вами в ближайшее время. Имя Телефон Отправить 0 Добавьте товары в желания 0 Добавьте товары для сравнения График работы: 12:00 - 19:00 0 Мой заказ ПАЛЬТО Все пальто Пальто демисезонные Пальто зимние женские Все зимние пальто Зимние пальто с мехом Кашемировые пальто Пальто с мехом Плащи Пальто с капюшоном НАТУРАЛЬНЫЕ ШУБЫ Все шубы Шубы норковые Все норковые шубы Полушубки норковые Норковые шубы с капюшоном Трансформеры из норки Элитная норка Норковые шубы больших размеров Шубы из голубой норки Шубы-автоледи из норки Шубы из мутона Все шубы из мутона Полушубки из мутона Полушубки Шубы из нутрии Все шубы из нутрии Трансформеры из нутрии Полушубки из нутрии Шубы из песца Все шубы из песца Трансформеры из песца Полушубок из песца Меховые кардиганы Шубы из бобрика (кролика рэкс) Все шубы из кролика рэкса Трансформеры из кролика Полушубки из бобрика (кролика рэкс) Трансформеры Шубы из кролика Все шубы из кролика Полушубки из кролика Шубы из куницы Все шубы из куницы Полушубки из куницы Шубы из каракуля Шубы из чернобурки Все шубы из чернобурки Полушубки из чернобурки Трансформеры из чернобурки Шубы из бобра Шубы из редких мехов Все шубы с эксклюзивного меха Трансформеры из редких мехов Полушубки из эксклюзивных мехов Шубы из лисы Все шубы из лисы Трансформеры из лисы Полушубки из лисы КУРТКИ Все куртки Дубленки натуральные Эко дубленки Зимние куртки Меховые куртки Куртки-дутики Куртки из экокожи ШУБЫ ИЗ ЭКО-МЕХА Шубы из эко-меха зимние Демисезонные искусственные шубы """

model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

# translate Arabic to English
tokenizer.src_lang = "ru_RU"
encoded_ar = tokenizer(article_ar, return_tensors="pt")
generated_tokens = model.generate(**encoded_ar, forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"])
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
# => "The Secretary-General of the United Nations says there is no military solution in Syria."

['Buy Natural Wood Shoes: Best Prices in Kiev and Ukraine | Chicly-Furs Рус Укр МАГАЗИН В К Киев Contacts About us Shipping and Payment Change and Return Tips Reviews Input +38 (067) 440 47 00 Call you back? Call you back? Enter your phone number and name. We will contact you in the near future. Name Phone Send 0 Add products to wishes 0 Add products for comparison Working time: 12:00 - 19:00 0 My order PALTO All collars De-season collars Winter collars Winter collars Winter collars Kashmir collars Leather collars Leather collars Natural Wood Shoes All Wood Shoes All Wood Shoes All Wood Shoes All Wood Shoes All Wood Shoes All Wood Shoes All Wood Shoes All Wood Shoes All Wood Shoes All Wood Shoes']

In [35]:
import dl_translate as dlt

mt = dlt.TranslationModel()  # Slow when you load it for the first time
#print(mt.available_languages())  # All languages that you can use
#print(mt.available_codes())  # Code corresponding to each language accepted
print(mt.get_lang_code_map())  # Dictionary of lang -> code

{'Afrikaans': 'af', 'Amharic': 'am', 'Arabic': 'ar', 'Asturian': 'ast', 'Azerbaijani': 'az', 'Bashkir': 'ba', 'Belarusian': 'be', 'Bulgarian': 'bg', 'Bengali': 'bn', 'Breton': 'br', 'Bosnian': 'bs', 'Catalan': 'ca', 'Valencian': 'ca', 'Cebuano': 'ceb', 'Czech': 'cs', 'Welsh': 'cy', 'Danish': 'da', 'German': 'de', 'Greek': 'el', 'English': 'en', 'Spanish': 'es', 'Estonian': 'et', 'Persian': 'fa', 'Fulah': 'ff', 'Finnish': 'fi', 'French': 'fr', 'Western Frisian': 'fy', 'Irish': 'ga', 'Gaelic': 'gd', 'Scottish Gaelic': 'gd', 'Galician': 'gl', 'Gujarati': 'gu', 'Hausa': 'ha', 'Hebrew': 'he', 'Hindi': 'hi', 'Croatian': 'hr', 'Haitian': 'ht', 'Haitian Creole': 'ht', 'Hungarian': 'hu', 'Armenian': 'hy', 'Indonesian': 'id', 'Igbo': 'ig', 'Iloko': 'ilo', 'Icelandic': 'is', 'Italian': 'it', 'Japanese': 'ja', 'Javanese': 'jv', 'Georgian': 'ka', 'Kazakh': 'kk', 'Khmer': 'km', 'Central Khmer': 'km', 'Kannada': 'kn', 'Korean': 'ko', 'Luxembourgish': 'lb', 'Letzeburgesch': 'lb', 'Ganda': 'lg', 'Linga

In [36]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import textwrap

def translate(text, src_lang="ru_RU", tgt_lang="en_XX", max_chunk_size=500):
    model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
    tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

    tokenizer.src_lang = src_lang
    translated_text = []

    # Split text into manageable chunks
    for chunk in textwrap.wrap(text, max_chunk_size):
        encoded = tokenizer(chunk, return_tensors="pt", truncation=True, max_length=1024)
        generated_tokens = model.generate(**encoded, forced_bos_token_id=tokenizer.lang_code_to_id[tgt_lang], max_length=1024)
        translated_text.append(tokenizer.decode(generated_tokens[0], skip_special_tokens=True))

    return " ".join(translated_text)

article_ru = """Купить Шубуиз Натурального Меха: Лучшие Цены в Киеве и Украине | Chicly-Furs Рус Укр МАГАЗИН В КИЕВЕ Контакты О нас Доставка и оплата Обмен и возврат Советы Отзывы Вход +38 (067) 440 47 00 Перезвонить вам? Перезвонить вам? Укажите ваш номер телефона и имя. Мы свяжемся с вами в ближайшее время. Имя Телефон Отправить 0 Добавьте товары в желания 0 Добавьте товары для сравнения График работы: 12:00 - 19:00 0 Мой заказ ПАЛЬТО Все пальто Пальто демисезонные Пальто зимние женские Все зимние пальто Зимние пальто с мехом Кашемировые пальто Пальто с мехом Плащи Пальто с капюшоном НАТУРАЛЬНЫЕ ШУБЫ Все шубы Шубы норковые Все норковые шубы Полушубки норковые Норковые шубы с капюшоном Трансформеры из норки Элитная норка Норковые шубы больших размеров Шубы из голубой норки Шубы-автоледи из норки Шубы из мутона Все шубы из мутона Полушубки из мутона Полушубки Шубы из нутрии Все шубы из нутрии Трансформеры из нутрии Полушубки из нутрии Шубы из песца Все шубы из песца Трансформеры из песца Полушубок из песца Меховые кардиганы Шубы из бобрика (кролика рэкс) Все шубы из кролика рэкса Трансформеры из кролика Полушубки из бобрика (кролика рэкс) Трансформеры Шубы из кролика Все шубы из кролика Полушубки из кролика Шубы из куницы Все шубы из куницы Полушубки из куницы Шубы из каракуля Шубы из чернобурки Все шубы из чернобурки Полушубки из чернобурки Трансформеры из чернобурки Шубы из бобра Шубы из редких мехов Все шубы с эксклюзивного меха Трансформеры из редких мехов Полушубки из эксклюзивных мехов Шубы из лисы Все шубы из лисы Трансформеры из лисы Полушубки из лисы КУРТКИ Все куртки Дубленки натуральные Эко дубленки Зимние куртки Меховые куртки Куртки-дутики Куртки из экокожи ШУБЫ ИЗ ЭКО-МЕХА Шубы из эко-меха зимние Демисезонные искусственные шубы """  # Full text here
translated_text = translate(article_ru)
print(translated_text)


Buy Shubuiz Natural Mecha: Best Prices in Kiev and Ukraine | Chicly-Furs Рус Укр МАГАЗИН В КЕЙВЕ Contacts About us Shipping and Payment Exchange and Return Tips Reviews Input +38 (067) 440 47 00 Call you? Call you? Enter your phone number and name. We will contact you soon. Name Phone Send 0 Add products to wishes 0 Add products for comparison Working time: 12:00 - 19:00 0 My order PALTO All trousers Demise trousers Winter trousers Women's winter trousers All winter trousers Winter Leather Pants Kashmiri Pants Leather Pants Leather Pants Capuchon Pants NATURAL SOUTH SOUTH SOUTH SOUTH SOUTH SOUTH SOUTH SOUTH SOUTH SOUTH SOUTH SOUTH SOUTH SOUTH SOUTH SOUTH SOUTH SOUTH SOUTH SOUTH SOUTH SOUTH SOUTH SOUTH from sand Wooden carnivals Wooden carnivals Wooden carnivals Wooden carnivals Wooden carnivals Wooden carnivals Wooden carnivals Wooden carnivals Wooden carnivals Wooden carnivals Wooden carnivals Wooden carnivals Wooden carnivals Wooden carnivals Wooden carnivals Wooden carnivals Wooden 

In [37]:
lang_map = {
    "da": "da_DK", "fr": "fr_XX", "es": "es_XX", "en": "en_XX", "ru": "ru_RU",
    "ja": "ja_XX", "ko": "ko_KR", "it": "it_IT", "zh-cn": "zh_CN", "lt": "lt_LT",
    "no": "no_XX", "de": "de_DE", "sl": "sl_SI", "nl": "nl_XX", "vi": "vi_VN",
    "th": "th_TH", "sv": "sv_SE", "ar": "ar_AR", "hr": "hr_HR", "el": "el_GR",
    "pt": "pt_XX", "id": "id_ID", "tr": "tr_TR", "he": "he_IL", "sk": "sk_SK",
    "ro": "ro_RO", "cs": "cs_CZ", "pl": "pl_PL", "hu": "hu_HU", "lv": "lv_LV",
    "et": "et_EE", "ca": "ca_XX", "fi": "fi_FI", "bg": "bg_BG", "uk": "uk_UA",
    "sq": "sq_AL", "mk": "mk_MK", "bn": "bn_IN"  # Bengali
}


In [38]:
data_1["mbart_lang_code"] = data_1["detected_language"].map(lang_map).fillna("en_XX")  # Default to English if unknown


In [39]:
Snipppet_3000_1=data_1.iloc[:3000,:]

In [ ]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import textwrap

def translate(text, src_lang, tgt_lang="en_XX", max_chunk_size=500):
    model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
    tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

    tokenizer.src_lang = src_lang
    translated_text = []

    # Split text into manageable chunks
    for chunk in textwrap.wrap(text, max_chunk_size):
        encoded = tokenizer(chunk, return_tensors="pt", truncation=True, max_length=1024)
        generated_tokens = model.generate(**encoded, forced_bos_token_id=tokenizer.lang_code_to_id[tgt_lang], max_length=1024)
        translated_text.append(tokenizer.decode(generated_tokens[0], skip_special_tokens=True))

    return " ".join(translated_text)


Snipppet_3000_1["translated_text"] = Snipppet_3000_1.apply(
    lambda row: row["normalized_text"] if row["mbart_lang_code"] == "en_XX" 
    else translate(row["normalized_text"], row["mbart_lang_code"]), axis=1
)


In [ ]:
Snipppet_3000_1.to_csv("Translated_3000_non.csv")

https://github.com/jfilter/clean-text/blob/main/README.md

https://stackoverflow.com/questions/68587883/python-translate-large-texts-to-english

https://github.com/xhluca/dl-translate/blob/main/README.md
